Imports

In [37]:
import astropy.units as u
import astropy.coordinates as coord
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import numpy as np
import gala.integrate as gi
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic
import scipy.optimize as opt
from scipy.optimize import minimize_scalar

Helper functions

In [38]:
def compare(x_vals1, y_vals1, x_vals2, y_vals2):
    pos1 = np.column_stack((x_vals1, y_vals1))
    mean1 = np.mean(pos1, axis=0)
    cov1 = np.cov(pos1, rowvar=False)

    pos2 = np.column_stack((x_vals2, y_vals2))
    mean2 = np.mean(pos2, axis=0)
    cov2 = np.cov(pos2, rowvar=False)

    dx = mean1[0] - mean2[0]
    dy = mean1[1] - mean2[1]

    combined_sigma_x = np.sqrt(np.max([cov1[0, 0],cov2[0, 0]]))
    combined_sigma_y = np.sqrt(np.max([cov1[1, 1],cov2[1, 1]]))

    Dx = dx / combined_sigma_x
    Dy = dy / combined_sigma_y

    return (Dx, Dy)

def final_points(dict, ics, num_steps):
    orbit = pot.integrate_orbit(ics, dt=-0.1*u.Myr, n_steps=num_steps)

    n_samples = 100
    ra_samples = np.random.normal(dict['ra'], dict['e_ra'], n_samples)
    dec_samples = np.random.normal(dict['dec'], dict['e_dec'], n_samples)
    dist_samples = np.random.normal(dict['dist'], dict['e_dist'], n_samples)
    pmra_samples = np.random.normal(dict['pmra'], dict['e_pmra'], n_samples)
    pmdec_samples = np.random.normal(dict['pmdec'], dict['e_pmdec'], n_samples)
    rv_samples = np.random.normal(dict['rv'], dict['e_rv'], n_samples)

    ics_list = []
    for ra, dec, dist, pmra, pmdec, rv in zip(ra_samples, dec_samples, dist_samples, pmra_samples, pmdec_samples, rv_samples):
        sc = coord.SkyCoord(ra=ra * u.degree,
                    dec=dec * u.degree,
                    distance=dist * u.kpc,
                    pm_ra_cosdec=pmra * (u.mas / u.yr),
                    pm_dec=pmdec * (u.mas / u.yr),
                    radial_velocity=rv * (u.km / u.s),
                    frame="icrs")
        gc = sc.transform_to(coord.Galactocentric)
        ics = gd.PhaseSpacePosition(pos=gc.cartesian.xyz, vel=gc.velocity.d_xyz)
        ics_list.append(ics)

    orbits = []
    orbits.append(orbit)
    for ics in ics_list:
        orbit = pot.integrate_orbit(ics, dt=-0.1*u.Myr, n_steps=num_steps)
        orbits.append(orbit)

    points = []
    for i, orbit in enumerate(orbits):
        final_pos = orbit[-1].pos.xyz.to(u.kpc).value[:3]
        points.append(final_pos)
    points = np.array(points)

    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]

    return x, y, z

def calculate_distance(num_steps, star, dwarf_dict, dwarf_ics):
    SkyCoord = coord.SkyCoord(
        ra=star['ra'] * u.degree,
        dec=star['dec'] * u.degree,
        distance=star['dist'] * u.kpc,
        pm_ra_cosdec=star['pmra'] * (u.mas / u.yr),
        pm_dec=star['pmdec'] * (u.mas / u.yr),
        radial_velocity=star['rv'] * (u.km / u.s),
        frame="icrs"
    )
    GC = SkyCoord.transform_to(coord.Galactocentric)
    ics = gd.PhaseSpacePosition(pos=GC.cartesian.xyz, vel=GC.velocity.d_xyz)

    dx, dy, dz = final_points(dwarf_dict, dwarf_ics, num_steps)
    x, y, z = final_points(star, ics, num_steps)

    XY = compare(dx, dy, x, y)
    distance_xy = np.sqrt(XY[0]**2 + XY[1]**2)
    XZ = compare(dx, dz, x, z)
    distance_xz = np.sqrt(XZ[0]**2 + XZ[1]**2)
    YZ = compare(dy, dz, y, z)
    distance_yz = np.sqrt(YZ[0]**2 + YZ[1]**2)
    total = np.sqrt(distance_xy**2 + distance_xz**2 + distance_yz**2)

    return total

Define potential

In [39]:
pot = gp.MilkyWayPotential()

Stars

In [40]:
hvs5 = {
    'ra': 139.4978105603, #degree
    'e_ra': 0.0001, #degree
    'dec': 67.3773227272, #degree
    'e_dec': 0.0001, #degree
    'dist': 44.20, #kpc
    'e_dist': 5.09, #kpc
    'pmra': 0.00, #mas/yr
    'e_pmra': 0.08, #mas/yr
    'pmdec': -0.99, #mas/yr
    'e_pmdec': 0.11, #mas/yr
    'rv': 545.50, #km/s
    'e_rv': 4.30, #km/s
    }
hvs8 = {
    'ra': 145.5584849227, #degree
    'e_ra': 0.0001, #degree
    'dec': 20.0561234065, #degree
    'e_dec': 0.0001, #degree
    'dist': 53.19, #kpc
    'e_dist': 9.80, #kpc
    'pmra': -0.88, #mas/yr
    'e_pmra': 0.16, #mas/yr
    'pmdec': -0.28, #mas/yr
    'e_pmdec': 0.14, #mas/yr
    'rv': 499.30, #km/s
    'e_rv': 2.90, #km/s
    }
hvs14 = {
    'ra': 161.0072805184, #degree
    'e_ra': 0.0001, #degree
    'dec': 6.1941762509, #degree
    'e_dec': 0.0001, #degree
    'dist': 102.66, #kpc
    'e_dist': 16.55, #kpc
    'pmra': -2.17, #mas/yr
    'e_pmra': 1.38, #mas/yr
    'pmdec': 2.28, #mas/yr
    'e_pmdec': 1.68, #mas/yr
    'rv': 537.30, #km/s
    'e_rv': 7.20, #km/s
    }
hvs17 = {
    'ra': 250.4849449351, #degree
    'e_ra': 0.0001, #degree
    'dec': 47.3961264077, #degree
    'e_dec': 0.0001, #degree
    'dist': 49.82, #kpc
    'e_dist': 3.90, #kpc
    'pmra': -1.13, #mas/yr
    'e_pmra': 0.09, #mas/yr
    'pmdec': -0.93, #mas/yr
    'e_pmdec': 0.10, #mas/yr
    'rv': 250.20, #km/s
    'e_rv': 2.90, #km/s
    }
hvs23 = {
    'ra': 329.1209186025, #degree
    'e_ra': 0.0001, #degree
    'dec': 0.9122807743, #degree
    'e_dec': 0.0001, #degree
    'dist': 114.87, #kpc
    'e_dist': 20.10, #kpc
    'pmra': -1.21, #mas/yr
    'e_pmra': 1.29, #mas/yr
    'pmdec': -2.46, #mas/yr
    'e_pmdec': 1.50, #mas/yr
    'rv': 259.30, #km/s
    'e_rv': 9.80, #km/s
    }

map = {
    0: 'hvs5',
    1: 'hvs8',
    2: 'hvs14',
    3: 'hvs17',
    4: 'hvs23',
}

Dwarf galaxy

In [41]:
# Leo I
dwarf_dict = {
    'ra': 152.117175, #degree
    'e_ra': 0.0001, #degree
    'dec': 12.3065, #degree
    'e_dec': 0.0001, #degree
    'dist': 254.0, #kpc
    'e_dist': 15.5, #kpc
    'pmra': -0.007, #mas/yr
    'e_pmra': 0.035, #mas/yr
    'pmdec': -0.119, #mas/yr
    'e_pmdec': 0.026, #mas/yr
    'rv': 282.9, #km/s
    'e_rv': 0.5, #km/s
    }
# # Sextans (I)
# dwarf_dict = {
#     'ra': 153.262584, #degree
#     'e_ra': 0.0001, #degree
#     'dec': -1.6147, #degree
#     'e_dec': 0.0001, #degree
#     'dist': 95.0, #kpc
#     'e_dist': 3.0, #kpc
#     'pmra': -0.41, #mas/yr
#     'e_pmra': 0.01, #mas/yr
#     'pmdec': 0.04, #mas/yr
#     'e_pmdec': 0.01, #mas/yr
#     'rv': 224.3, #km/s
#     'e_rv': 0.1, #km/s
#     }
dwarf = coord.SkyCoord(
    ra=dict['ra'] * u.degree,
    dec=dict['dec'] * u.degree,
    distance=dict['dist'] * u.kpc,
    pm_ra_cosdec=dict['pmra'] * (u.mas / u.yr),
    pm_dec=dict['pmdec'] * (u.mas / u.yr),
    radial_velocity=dict['rv'] * (u.km / u.s),
    frame="icrs"
)
dwarf_GC = dwarf.transform_to(coord.Galactocentric)
dwarf_ics = gd.PhaseSpacePosition(pos=dwarf_GC.cartesian.xyz, vel=dwarf_GC.velocity.d_xyz)


        Use name instead. [gala.dynamics.core]


Calculate

In [43]:
for i, star in enumerate([hvs5, hvs8, hvs14, hvs17, hvs23]):
    result = minimize_scalar(
        calculate_distance,
        args=(star, dwarf_dict, dwarf_ics),
        bounds=(500, 15000),  # search between 500 and 15000 steps
        method='bounded'
    )

    print(f"{map[i]}: minimum distance {result.fun} at {result.x} steps")

        Use name instead. [gala.dynamics.core]


hvs5: minimum distance 20.340880126284755 at 6044.877940737433 steps
hvs8: minimum distance 12.632298064239913 at 13286.066549983396 steps
hvs14: minimum distance 2.8330165533618072 at 9463.436712209297 steps
hvs17: minimum distance 13.892170070230996 at 9461.492195065215 steps
hvs23: minimum distance 2.553366207511253 at 9507.480174422486 steps
